# JobSessionManager

> Manage job-related metadata that needs to be accessed across multiple requests.

In [ ]:
#| default_exp core.job_session

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, Optional

## JobSessionManager Class

The `JobSessionManager` bridges the gap between job execution (managed by job managers) and presentation logic (in route handlers) by storing display-related metadata in the user's session.

This is useful when:
- Job managers handle execution but don't track UI-specific data
- You need to preserve user-selected options for displaying job results
- Multiple routes need to access the same job display information

In [ ]:
#| export
class JobSessionManager:
    """Manage job-related metadata that needs to be accessed across multiple requests."""

    def __init__(
        self, 
        sess:Any, # FastHTML session object
        prefix:str="job" # Prefix for job keys in session
    ):
        """Initialize job session manager."""
        self.sess = sess
        self.prefix = prefix

    def _make_job_key(
        self, 
        job_id:str # Unique job identifier
    ) -> str: # Session key for job metadata
        """Create session key for job metadata."""
        return f"{self.prefix}_{job_id}"

    def store_job_metadata(
        self, 
        job_id:str, # Unique job identifier
        metadata:Dict[str, Any] # Dictionary of metadata to store
    ) -> None:
        """Store metadata for a job."""
        self.sess[self._make_job_key(job_id)] = metadata

    def get_job_metadata(
        self, 
        job_id:str, # Unique job identifier
        default:Optional[Dict[str, Any]]=None # Default value if metadata not found
    ) -> Dict[str, Any]: # Job metadata dictionary
        """Retrieve metadata for a job."""
        if default is None:
            default = {}
        return self.sess.get(self._make_job_key(job_id), default)

    def delete_job_metadata(
        self, 
        job_id:str # Unique job identifier
    ) -> None:
        """Remove metadata for a job."""
        key = self._make_job_key(job_id)
        if key in self.sess:
            del self.sess[key]

    def clear_all_jobs(self) -> None:
        """Clear all job metadata from session."""
        keys_to_delete = [k for k in self.sess.keys() if k.startswith(f"{self.prefix}_")]
        for key in keys_to_delete:
            del self.sess[key]

## Usage Examples

These examples show how to use `JobSessionManager` to store and retrieve job-related metadata across different requests.

In [ ]:
# Create a mock session
mock_session = {}

# Create job session manager
job_sess = JobSessionManager(mock_session)
job_sess

<__main__.JobSessionManager>

In [ ]:
# Store metadata for a transcription job
job_sess.store_job_metadata("job-123", {
    "file_info": {
        "name": "interview.mp3",
        "path": "/media/audio/interview.mp3",
        "size": 1024000
    },
    "plugin_info": {
        "id": "whisper_large",
        "title": "Whisper Large",
        "version": "3.0"
    },
    "user_preferences": {
        "language": "en",
        "format": "srt"
    }
})

# Show the session
mock_session

{'job_job-123': {'file_info': {'name': 'interview.mp3',
   'path': '/media/audio/interview.mp3',
   'size': 1024000},
  'plugin_info': {'id': 'whisper_large',
   'title': 'Whisper Large',
   'version': '3.0'},
  'user_preferences': {'language': 'en', 'format': 'srt'}}}

In [ ]:
# Retrieve job metadata
metadata = job_sess.get_job_metadata("job-123")
print(f"File: {metadata['file_info']['name']}")
print(f"Plugin: {metadata['plugin_info']['title']}")
print(f"Language: {metadata['user_preferences']['language']}")

File: interview.mp3
Plugin: Whisper Large
Language: en


In [ ]:
# Store metadata for multiple jobs
job_sess.store_job_metadata("job-456", {
    "file_info": {"name": "podcast.mp3"},
    "plugin_info": {"title": "Faster Whisper"}
})

job_sess.store_job_metadata("job-789", {
    "file_info": {"name": "lecture.mp4"},
    "plugin_info": {"title": "Voxtral"}
})

print("Session now contains multiple jobs:")
mock_session

Session now contains multiple jobs:


{'job_job-123': {'file_info': {'name': 'interview.mp3',
   'path': '/media/audio/interview.mp3',
   'size': 1024000},
  'plugin_info': {'id': 'whisper_large',
   'title': 'Whisper Large',
   'version': '3.0'},
  'user_preferences': {'language': 'en', 'format': 'srt'}},
 'job_job-456': {'file_info': {'name': 'podcast.mp3'},
  'plugin_info': {'title': 'Faster Whisper'}},
 'job_job-789': {'file_info': {'name': 'lecture.mp4'},
  'plugin_info': {'title': 'Voxtral'}}}

In [ ]:
# Get metadata for non-existent job with default
missing_metadata = job_sess.get_job_metadata("job-999", default={"error": "Not found"})
missing_metadata

{'error': 'Not found'}

In [ ]:
# Delete specific job metadata
job_sess.delete_job_metadata("job-456")
print("After deleting job-456:")
mock_session

After deleting job-456:


{'job_job-123': {'file_info': {'name': 'interview.mp3',
   'path': '/media/audio/interview.mp3',
   'size': 1024000},
  'plugin_info': {'id': 'whisper_large',
   'title': 'Whisper Large',
   'version': '3.0'},
  'user_preferences': {'language': 'en', 'format': 'srt'}},
 'job_job-789': {'file_info': {'name': 'lecture.mp4'},
  'plugin_info': {'title': 'Voxtral'}}}

In [ ]:
# Clear all job metadata
job_sess.clear_all_jobs()
print("After clearing all jobs:")
mock_session

After clearing all jobs:


{}

In [ ]:
# Example: Using with custom prefix
processing_sess = JobSessionManager({}, prefix="processing")
processing_sess.store_job_metadata("task-001", {"status": "running"})

# Notice the different key prefix
processing_sess.sess

{'processing_task-001': {'status': 'running'}}

## Real-World Usage Pattern

Here's a typical pattern showing how job metadata flows through a multi-step process:

In [ ]:
# Simulating a workflow across multiple routes
session = {}
job_manager = JobSessionManager(session)

# Step 1: User starts a job
def start_job_route(job_id, file_name, plugin_title):
    job_manager.store_job_metadata(job_id, {
        "file_info": {"name": file_name},
        "plugin_info": {"title": plugin_title},
        "started_at": "2025-01-01T00:00:00"
    })
    print(f"Job {job_id} started with {file_name}")

# Step 2: Monitor job progress
def monitor_job_route(job_id):
    metadata = job_manager.get_job_metadata(job_id)
    if metadata:
        print(f"Monitoring job {job_id}: {metadata['file_info']['name']} using {metadata['plugin_info']['title']}")
    else:
        print(f"Job {job_id} not found")

# Step 3: Display results
def show_results_route(job_id):
    metadata = job_manager.get_job_metadata(job_id)
    if metadata:
        print(f"Results for {metadata['file_info']['name']}")
        print(f"Processed by: {metadata['plugin_info']['title']}")
        print(f"Started: {metadata['started_at']}")

# Simulate the workflow
print("=== Simulated Workflow ===")
start_job_route("job-abc", "meeting.mp3", "Whisper")
monitor_job_route("job-abc")
show_results_route("job-abc")
print("\nSession state:")
session

=== Simulated Workflow ===
Job job-abc started with meeting.mp3
Monitoring job job-abc: meeting.mp3 using Whisper
Results for meeting.mp3
Processed by: Whisper
Started: 2025-01-01T00:00:00

Session state:


{'job_job-abc': {'file_info': {'name': 'meeting.mp3'},
  'plugin_info': {'title': 'Whisper'},
  'started_at': '2025-01-01T00:00:00'}}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()